# Import Packages

In [1]:
!pip3 install numpy
!pip3 install pandas
!pip3 install geopandas
!pip3 install regex
!pip3 install unidecode
!pip3 install nltk
!pip3 install scikit-learn
!pip3 install matplotlib
!pip3 install seaborn
!pip3 install wordcloud
!pip3 install folium
!pip3 install branca
!pip3 install bs4

In [1]:
# Data Structures
import numpy  as np
import pandas as pd
import json

# Text Processing
import re
import nltk.corpus
import nltk.text
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from unidecode import unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

# K-Means
from sklearn import cluster
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

# Visualization and Analysis
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from sklearn.metrics import silhouette_samples, silhouette_score
from wordcloud import WordCloud

# Map Viz
import folium
#import branca.colormap as cm
from branca.element import Figure

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\selam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\selam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Read CSV into a Dataframe


In [2]:
import pandas as pd
dataset = pd.read_csv("HazardInsurance.csv")
dataset.head()

,uuid,ll_doc_name_reformatted,enc_loan_guid,ll_doc_name,ll_doc_start_page,ll_doc_stop_page,ocr_doc_page_num,ocr_loan_pacakge_page_num,ocr_json_payload,ocr_text,...,enc_nondelind,enc_clientid,enc_clientname,enc_pnmac_adaptor_flag,enc_newurla,ll_doc_category,ll_doc_order,year,month,day
0,65ae2aa6-a47d-4a95-9a4e-7cbb8f096456,Hazard Insurance,0dc04ae9-52d1-4fe3-b0aa-2b842bb5d18e,hazard_insurance,539,542,1,540,"[{""Key"": ""Total Living Area:"", ""Value"": ""2092 ...","['07/19/21 16:36:59 800-776-4737', 'Veterans U...",...,N,700354,"MORTGAGE RESEARCH CENTER, LLC",True,Y,piti,15,2022,5,26
1,736cd728-267e-4d31-b5d0-1b604716a720,Hazard Insurance,0dc04ae9-52d1-4fe3-b0aa-2b842bb5d18e,hazard_insurance,539,542,2,541,"[{""Key"": ""Perimeter:"", ""Value"": ""Rectangular o...","['07/19/21 16:37:33 800-776-4737', 'Veterans U...",...,N,700354,"MORTGAGE RESEARCH CENTER, LLC",True,Y,piti,15,2022,5,26
2,abf65e4e-c042-4672-a4fb-7ffcdccd6f4f,Hazard Insurance,0dc04ae9-52d1-4fe3-b0aa-2b842bb5d18e,hazard_insurance,539,542,3,542,"[{""row_index"": 1, ""column_index"": 1, ""text"": ""...","['07/19/21 16:38:06 800-776-4737', 'Veterans U...",...,N,700354,"MORTGAGE RESEARCH CENTER, LLC",True,Y,piti,15,2022,5,26
3,78e891a1-d75d-4e1b-8e7f-bb1cb577d87f,Hazard Insurance,1adfea14-b1f6-4549-85e5-ec1ac5e4bf8c,hazard_insurance,212,222,1,213,"[{""Key"": ""Exterior wall type:"", ""Value"": ""100%...","['Renewal House & Home Policy Declarations', '...",...,N,701346,THE FEDERAL SAVINGS BANK,True,Y,piti,15,2022,5,26
4,91445cb8-7ce2-4d1b-9257-6dff6cd2ad99,Hazard Insurance,1adfea14-b1f6-4549-85e5-ec1ac5e4bf8c,hazard_insurance,212,222,2,214,"[{""Key"": ""Policy number:"", ""Value"": ""807 842 4...","['Renewal House & Home Policy Declarations', '...",...,N,701346,THE FEDERAL SAVINGS BANK,True,Y,piti,15,2022,5,26


## Document Assignments
Creation of a dictionary with the key being the doc package ID and the value assignment is all of the OCR Text associated with said document.

In [3]:
values = {}
for index, row in dataset.iterrows():
    new_lst = []
    if row['enc_loan_guid'] in values.keys():
        values[row['enc_loan_guid']].append(row['ocr_text'])
    else:
        new_lst.append(row['ocr_text'])
        values[row['enc_loan_guid']] = new_lst

# Text Processing
## 1. Whitelisting
Utilizes BeautifulSoup package to retreive list of US Insurance companies. This list will leave only strings that are insurance company names.

In [4]:
# import required modules
from bs4 import BeautifulSoup
import requests
import random

# get URL
page = requests.get("https://en.wikipedia.org/wiki/List_of_United_States_insurance_companies")

# scrape webpage
soup = BeautifulSoup(page.content, 'html.parser')
object = soup.find(id="mw-content-text")

links = object.find_all("a")
titles = []
print(len(links))
for link in links:
    title = link.get('title')
    if title == 'Edit section: Life and annuity':
        break
    if title != None:
        titles.append(title)
titles.remove('Insurance')
titles.remove('Life insurance')
print(titles)

438
['Philadelphia Contributionship', 'New York (state)', 'Great Chicago Fire', 'Edit section: U.S. insurance companies', 'Wikipedia:WikiProject Lists', 'Edit section: Property and casualty insurance', '21st Century Insurance', 'Acuity Insurance', 'Aflac', 'Allianz Life', 'Allied Insurance', 'Allstate', 'American Automobile Association', 'American Family Insurance', 'American Income Life Insurance Company', 'American International Group', 'American National Insurance Company', 'American Strategic Insurance', 'Ameriprise Auto & Home Insurance', 'Ameriprise Financial', 'Ameritas Life Insurance Company', 'Amica Mutual Insurance', 'AmTrust Financial Services', 'Applied Underwriters', 'Arbella Insurance Group', 'Assurant', 'Assurity Life Insurance Company', 'Auto-Owners Insurance', 'AXA Equitable Life Insurance Company', 'Bankers Life and Casualty Company', 'Berkshire Hathaway', 'Brotherhood Mutual Insurance Company', 'CareSource', 'Chubb Corp', 'Citizens Property Insurance Corporation', 'C

## 2. The main text processing function.
A section back, at the exploration of our dataset, we noticed some words containg weird characters that should be removed. By using RegEx our main processing function will remove unknown ASCII symbols, especial chars, numbers, e-mails, URLs, etc

In [5]:
def processText(text):
  text= text.replace(u'\ufffd', '8')   # Replaces the ASCII '�' symbol with '8'
  text = text.replace(',', '')          # Removes commas
  text = text.rstrip('\n')              # Removes line breaks
  text = text.casefold()                # Makes all letters lowercase

  text = re.sub('\W_',' ', text)        # removes specials characters and leaves only words
  text= re.sub("\S*\d\S*"," ", text)   # removes numbers and words concatenated with numbers IE h4ck3r. Removes road names such as BR-381.
  text= re.sub("\S*@\S*\s?"," ", text) # removes emails and mentions (words with @)
  text= re.sub(r'http\S+', '', text)   # removes URLs with http
  text= re.sub(r'www\S+', '', text)    # removes URLs with www

  keep = False

  for t in titles:
      if t.casefold() in text:
          text = t.casefold()
          keep =True

  if keep == False:
      text= 'missing'

  return text

### Creation of Company Dictionary
This dictionary will serve as refrence to the insurance company for each document package

In [6]:
companies = {'missing': []}
for key in values.keys():
  for val in values[key]:
    result = processText(val)
    if result != 'missing' and result != 'workers\' compensation':
      new_lst = []
      if result in companies.keys():
        companies[result].append(key)
        break
      else:
        new_lst.append(key)
        companies[result] = new_lst

  if type(values[key]) is list:
    companies['missing'].append(key)

## Corpus Processing
### 1. Stop Words
We will do a data engineering routine with our anthems dataset so later we can make a good statistical model. In order to do so, we'll remove all words that don't contribute to the semantic meaning of the text (words that are not within the english alphabet) and keep all of the remaining words.

In [38]:
def removeWords(listOfTokens, listOfWords):
    return [token for token in listOfTokens if not token.lower() in listOfWords]


def twoLetters(listOfTokens):
    twoLetterWord = []
    for token in listOfTokens:
        if len(token) <= 2 or len(token) >= 21:
            twoLetterWord.append(token)
    return twoLetterWord

def stop(word, lis):
    word = word.split()
    for w in lis:
        if w in word:
            word.remove(w)
    return ' '.join(word)

### 3.The main corpus processing function.
A section back, at the exploration of our dataset, we noticed some words containg weird characters that should be removed. By using RegEx our main processing function will remove unknown ASCII symbols, especial chars, numbers, e-mails, URLs, etc. It also uses the auxiliary funcitions defined above.



In [39]:
def processCorpus(corpus, language):
    stopwords = nltk.corpus.stopwords.words(language)
    common_words = ['insurance', 'policy', 'coverage', 'date', 'may', 'please', 'page', 'property', 'loss', 'the', 'this', 'address']

    for document in corpus:
        index = corpus.index(document)
        corpus[index] = corpus[index].replace(u'\ufffd', '8')   # Replaces the ASCII '�' symbol with '8'
        corpus[index] = corpus[index].replace(',', '')          # Removes commas
        corpus[index] = corpus[index].rstrip('\n')              # Removes line breaks
        corpus[index] = corpus[index].casefold()                # Makes all letters lowercase
        corpus[index] = stop(corpus[index], common_words)
        corpus[index] = stop(corpus[index], stopwords)

        corpus[index] = re.sub('\W_',' ', corpus[index])        # removes specials characters and leaves only words
        corpus[index] = re.sub("\S*\d\S*"," ", corpus[index])   # removes numbers and words concatenated with numbers IE h4ck3r. Removes road names such as BR-381.
        corpus[index] = re.sub("\S*@\S*\s?"," ", corpus[index]) # removes emails and mentions (words with @)
        corpus[index] = re.sub(r'http\S+', '', corpus[index])   # removes URLs with http
        corpus[index] = re.sub(r'www\S+', '', corpus[index])    # removes URLs with www

    return corpus

### Cluster Analysis
Now we can choose the best number of K and take a deeper look at each cluster.

In [40]:
def cluster_pop(docs,k):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(docs)
    # Perform clustering using K-means algorithm
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    # Get cluster labels for each document
    cluster_labels = kmeans.labels_
    # Get the number of documents within each cluster
    cluster_sizes = [len(cluster_labels[cluster_labels == i]) for i in range(k)]
    # Get the top 10 words in each cluster
    top_words_per_cluster = []
    order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names_out()
    for i in range(k):
        top_words = [terms[ind] for ind in order_centroids[i, :10]]
        top_words_per_cluster.append(top_words)
    # Calculate confidence scores based on distances to cluster centroids
    distances = pairwise_distances_argmin_min(X, kmeans.cluster_centers_)
    min_distances = distances[1]
    max_distance = max(min_distances)
    confidence_scores = [1 - (distance / max_distance) for distance in min_distances]
    # Print cluster information
    for i in range(k):
        results.write(f"Cluster {i}:\n")
        results.write(f"Number of documents: {cluster_sizes[i]}\n")
        results.write(f"Top 10 words: {', '.join(top_words_per_cluster[i])}\n")
        results.write(f"Confidence score: {confidence_scores[i]}\n")

In [41]:
results = open('results.txt', 'w')
for key in companies.keys():
    results.write("The insurance company is {}\n".format(key))
    txt = []
    for doc in companies[key]:
        txt.append(" ".join(values[doc]))
    txt = processCorpus(txt, 'english')
    try:
        cluster_pop(txt, 3)
    except:
        try:
            cluster_pop(txt, 2)
        except:
            try:
                cluster_pop(txt, 1)
            except:
                results.write("CLUSTER FAIL")
    results.write("------------------------------------------\n ")

\\pnmac.com\profiles\Users\selam.V2\Desktop\IDP\.venv\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
\\pnmac.com\profiles\Users\selam.V2\Desktop\IDP\.venv\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
\\pnmac.com\profiles\Users\selam.V2\Desktop\IDP\.venv\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
\\pnmac.com\profiles\Users\selam.V2\Desktop\IDP\.venv\lib\site-packages\sklearn\cluster\